<a href="https://colab.research.google.com/github/kumar045/Assignment/blob/main/Pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
import os
from urllib.parse import unquote

def download_fachinformation(url_template, token, output_filename):
    """
    Lädt die Fachinformation herunter, indem der Token in die URL eingesetzt wird.

    :param url_template: Die URL aus dem XML (enthält {Anmeldetoken})
    :param token: Ihr persönlicher Webservice-Token vom APOVERLAG
    :param output_filename: Pfad, wo das PDF gespeichert werden soll
    """

    # 1. URL dekodieren falls nötig und Platzhalter ersetzen
    url = unquote(url_template)
    download_url = url.replace('{Anmeldetoken}', token)

    # Falls die URL noch encoded ist
    download_url = download_url.replace('%7BAnmeldetoken%7D', token)

    print(f"Lade herunter von: {download_url}")

    try:
        # 2. HTTP GET Request mit erweiterten Parametern
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

        response = requests.get(
            download_url,
            stream=True,
            timeout=30,
            headers=headers,
            allow_redirects=True
        )

        # Prüfen, ob der Abruf erfolgreich war
        if response.status_code == 200:
            # Content-Type prüfen (sollte PDF sein)
            content_type = response.headers.get('content-type', '')
            if 'pdf' not in content_type.lower() and 'application/octet-stream' not in content_type.lower():
                print(f"Warnung: Unerwarteter Content-Type: {content_type}")

            # 3. Verzeichnis erstellen falls nötig
            output_dir = os.path.dirname(output_filename)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # 4. Datei speichern
            with open(output_filename, 'wb') as pdf_file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:  # Filter out keep-alive chunks
                        pdf_file.write(chunk)

            file_size = os.path.getsize(output_filename)
            print(f"✓ Erfolg! Datei gespeichert unter: {output_filename}")
            print(f"  Dateigröße: {file_size:,} Bytes ({file_size/1024:.1f} KB)")

        else:
            print(f"✗ Fehler beim Download. HTTP Status: {response.status_code}")
            print(f"  Server-Antwort: {response.text[:500]}")  # Erste 500 Zeichen

    except requests.exceptions.Timeout:
        print("✗ Timeout: Server hat nicht rechtzeitig geantwortet")
    except requests.exceptions.ConnectionError:
        print("✗ Verbindungsfehler: Server nicht erreichbar")
    except requests.exceptions.RequestException as e:
        print(f"✗ Fehler beim Download: {e}")
    except IOError as e:
        print(f"✗ Fehler beim Speichern der Datei: {e}")

# --- ANWENDUNGSBEISPIEL ---

if __name__ == "__main__":
    # 1. Ihr echter Token (aus dem APOVERLAG Download-Bereich -> Passwortverwaltung)
    MY_TOKEN = "NI%2BFzeQS4ni6iAFio16kgQ%3D%3D"

    # 2. Beispiel-URL aus dem XML (Feld: <Fachinformation>)
    xml_url = "https://services.apoverlag.at/pdfs_svc/Streaming.svc/getFIDocument?dt=1&tk={Anmeldetoken}&pi=0-12345&ac=true"

    # 3. Token-Validierung
    if MY_TOKEN == "HIER_IHREN_TOKEN_EINFUEGEN":
        print("⚠ Bitte setzen Sie zuerst die Variable MY_TOKEN im Skript!")
        print("  Token erhalten Sie im APOVERLAG Download-Bereich unter Passwortverwaltung")
    else:
        # Ausgabedatei (kann auch Pfad enthalten, z.B. "output/Fachinformation.pdf")
        output_file = "Fachinformation_12345.pdf"
        download_fachinformation(xml_url, MY_TOKEN, output_file)

Lade herunter von: https://services.apoverlag.at/pdfs_svc/Streaming.svc/getFIDocument?dt=1&tk=NI%2BFzeQS4ni6iAFio16kgQ%3D%3D&pi=0-12345&ac=true
✓ Erfolg! Datei gespeichert unter: Fachinformation_12345.pdf
  Dateigröße: 604,578 Bytes (590.4 KB)


In [9]:
import requests
import os
from urllib.parse import unquote, urlencode

def download_fachinformation(token, zulassungsnummer=None, pharmazentralnummer=None,
                            document_type='fi', output_filename=None):
    """
    Lädt die Fachinformation vom APOVERLAG herunter.

    :param token: Ihr persönlicher Webservice-Token vom APOVERLAG
    :param zulassungsnummer: Zulassungsnummer (pi-Parameter), z.B. "1-12345"
    :param pharmazentralnummer: Pharmazentralnummer (ai-Parameter), z.B. "12345"
    :param document_type: Dokumenttyp (dt-Parameter)
        - 'fi' oder '1' = Fachinformation
        - 'gb' oder '2' = Gebrauchsinformation
        - 'sa' oder '3' = Zusammenfassung der Merkmale
    :param output_filename: Pfad für die Ausgabedatei (optional)
    """

    # Dokumenttyp-Mapping
    doc_types = {
        'fi': '1', 'fachinformation': '1', '1': '1',
        'gb': '2', 'gebrauchsinformation': '2', '2': '2',
        'sa': '3', 'zusammenfassung': '3', '3': '3'
    }

    dt = doc_types.get(document_type.lower(), '1')

    # Validierung
    if not zulassungsnummer and not pharmazentralnummer:
        print("✗ Fehler: Sie müssen entweder eine Zulassungsnummer (pi) ODER eine Pharmazentralnummer (ai) angeben!")
        return False

    # Base URL
    base_url = "https://services.apoverlag.at/pdfs_svc/Streaming.svc/getFIDocument"

    # Parameter zusammenstellen
    params = {
        'dt': dt,
        'tk': token,
        'ac': 'true'
    }

    if zulassungsnummer:
        params['pi'] = zulassungsnummer
    if pharmazentralnummer:
        params['ai'] = pharmazentralnummer

    # URL konstruieren
    download_url = f"{base_url}?{urlencode(params)}"

    print(f"Abrufen von APOVERLAG...")
    print(f"  Dokumenttyp: {document_type.upper()}")
    if zulassungsnummer:
        print(f"  Zulassungsnummer: {zulassungsnummer}")
    if pharmazentralnummer:
        print(f"  Pharmazentralnummer: {pharmazentralnummer}")

    try:
        # HTTP GET Request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

        response = requests.get(
            download_url,
            stream=True,
            timeout=30,
            headers=headers,
            allow_redirects=True
        )

        # Prüfen ob erfolgreich
        if response.status_code == 200:
            # Content-Type prüfen
            content_type = response.headers.get('content-type', '')

            # Prüfen ob HTML-Fehlerseite statt PDF
            if 'text/html' in content_type.lower():
                print("✗ Fehler: Server hat HTML statt PDF zurückgegeben")
                print("  Mögliche Ursachen:")
                print("  • Ungültige Zulassungsnummer oder Pharmazentralnummer")
                print("  • Token ist abgelaufen oder ungültig")
                print("  • Dokument existiert nicht für diese Nummer")
                return False

            # Dateiname generieren falls nicht angegeben
            if not output_filename:
                identifier = zulassungsnummer or pharmazentralnummer
                identifier = identifier.replace('/', '-').replace('\\', '-')
                output_filename = f"APOVERLAG_{document_type}_{identifier}.pdf"

            # Verzeichnis erstellen falls nötig
            output_dir = os.path.dirname(output_filename)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Datei speichern
            with open(output_filename, 'wb') as pdf_file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        pdf_file.write(chunk)

            file_size = os.path.getsize(output_filename)
            print(f"✓ Erfolg! Datei gespeichert unter: {output_filename}")
            print(f"  Dateigröße: {file_size:,} Bytes ({file_size/1024:.1f} KB)")
            return True

        else:
            print(f"✗ Fehler beim Download. HTTP Status: {response.status_code}")
            if response.status_code == 404:
                print("  Dokument nicht gefunden!")
            print(f"  Server-Antwort: {response.text[:500]}")
            return False

    except requests.exceptions.Timeout:
        print("✗ Timeout: Server hat nicht rechtzeitig geantwortet")
        return False
    except requests.exceptions.ConnectionError:
        print("✗ Verbindungsfehler: Server nicht erreichbar")
        return False
    except requests.exceptions.RequestException as e:
        print(f"✗ Fehler beim Download: {e}")
        return False
    except IOError as e:
        print(f"✗ Fehler beim Speichern der Datei: {e}")
        return False

# --- ANWENDUNGSBEISPIELE ---

if __name__ == "__main__":
    # Ihr echter Token
    MY_TOKEN = "NI%2BFzeQS4ni6iAFio16kgQ%3D%3D"

    if MY_TOKEN == "HIER_IHREN_TOKEN_EINFUEGEN":
        print("⚠ WICHTIG: Bitte setzen Sie zuerst die Variable MY_TOKEN!")
        print("  Token erhalten Sie im APOVERLAG Download-Bereich unter Passwortverwaltung")
        print()
    else:
        print("=" * 60)
        print("APOVERLAG Fachinformation Download")
        print("=" * 60)
        print()

        # Beispiel 1: Mit Zulassungsnummer
        print("Beispiel 1: Download mit Zulassungsnummer")
        print("-" * 60)
        download_fachinformation(
            token=MY_TOKEN,
            zulassungsnummer="1-12345",  # Ihre echte Zulassungsnummer
            document_type='fi'
        )
        print()

        # Beispiel 2: Mit Pharmazentralnummer
        print("Beispiel 2: Download mit Pharmazentralnummer")
        print("-" * 60)
        download_fachinformation(
            token=MY_TOKEN,
            pharmazentralnummer="12345",  # Ihre echte Pharmazentralnummer
            document_type='fi',
            output_filename="mein_dokument.pdf"
        )
        print()

        # Beispiel 3: Gebrauchsinformation
        print("Beispiel 3: Gebrauchsinformation statt Fachinformation")
        print("-" * 60)
        download_fachinformation(
            token=MY_TOKEN,
            zulassungsnummer="1-12345",
            document_type='gb'  # 'gb' für Gebrauchsinformation
        )
        print()

    print()
    print("HINWEISE:")
    print("• Zulassungsnummer (pi): Format meist '1-12345' oder '0-12345'")
    print("• Pharmazentralnummer (ai): Reine Zahlenkombination")
    print("• Dokumenttypen: 'fi' (Fachinformation), 'gb' (Gebrauchsinformation), 'sa' (Zusammenfassung)")
    print("• Prüfen Sie die Nummern in Ihrem XML-Dokument vom APOVERLAG")

APOVERLAG Fachinformation Download

Beispiel 1: Download mit Zulassungsnummer
------------------------------------------------------------
Abrufen von APOVERLAG...
  Dokumenttyp: FI
  Zulassungsnummer: 1-12345
✓ Erfolg! Datei gespeichert unter: APOVERLAG_fi_1-12345.pdf
  Dateigröße: 604,578 Bytes (590.4 KB)

Beispiel 2: Download mit Pharmazentralnummer
------------------------------------------------------------
Abrufen von APOVERLAG...
  Dokumenttyp: FI
  Pharmazentralnummer: 12345
✗ Fehler beim Download. HTTP Status: 400
  Server-Antwort: {"type":"https://tools.ietf.org/html/rfc7231#section-6.5.1","title":"One or more validation errors occurred.","status":400,"traceId":"00-35f7e3e791b3f19450822d9131ec733a-96758248e2ef0854-00","errors":{"pi":["The pi field is required."]}}

Beispiel 3: Gebrauchsinformation statt Fachinformation
------------------------------------------------------------
Abrufen von APOVERLAG...
  Dokumenttyp: GB
  Zulassungsnummer: 1-12345
✓ Erfolg! Datei gespeichert